# 1367. 使网格图至少有一条有效路径的最小代价

这道题目（1368. 使网格图至少有一条有效路径的最小代价）的核心是**将网格路径问题转化为“最短路径问题”**，并利用**Dijkstra算法**求解最小代价。下面从题目分析、思路转化、算法选择到代码实现，一步步详细讲解。

## 一、题目核心需求分析

我们需要解决的问题可以概括为：  
在一个`m x n`的网格中，每个格子有一个"默认方向"（1-4分别对应右、左、下、上）。从左上角`(0,0)`出发，按格子的默认方向移动，若能到达右下角`(m-1, n-1)`，则代价为0；若不能，可修改格子的方向（每次修改花费1，每个格子只能改一次），求让路径有效的**最小总代价**。

## 二、问题转化：从网格到"带权图"

要解决这个问题，关键是将网格转化为"图论中的带权图"，具体转化规则如下：

1. **节点定义**：网格中的每个格子`(x, y)`视为图中的一个"节点"。  
2. **边的定义**：每个节点`(x, y)`可以向**上下左右四个方向**的相邻格子（`(x±1,y)`或`(x,y±1)`）移动，这种移动视为一条"边"。  
3. **边的权重（代价）**：  
   - 若移动方向与当前格子`(x,y)`的默认方向一致（例如格子默认方向是"右"，且我们恰好向右移动），则无需修改，代价为`0`；  
   - 若移动方向与默认方向不一致（例如格子默认方向是"右"，但我们需要向左移动），则需要修改格子方向，代价为`1`。  

## 三、为什么用Dijkstra算法？

转化为带权图后，问题变成了：**从起点`(0,0)`到终点`(m-1, n-1)`的"最短路径"是什么？** 这里的"最短"指的是路径上所有边的权重之和（总代价）最小。

Dijkstra算法的核心是**求"非负权图"中从起点到终点的最短路径**，而本题中边的权重只有`0`或`1`（均为非负数），完全满足Dijkstra算法的适用条件。因此，我们可以用Dijkstra算法求解。

## 四、算法思路拆解

Dijkstra算法的核心思想是**"贪心+优先队列"**：每次从"未确定最短路径的节点"中，选择当前代价最小的节点进行处理，并更新其相邻节点的代价，直到到达终点。

对应到本题，具体步骤如下：

1. **初始化状态**：  
   - 记录每个格子`(x,y)`的最小代价（初始为`-1`，表示未访问）。  
   - 用**优先队列（最小堆）** 存储待处理的节点，队列中元素为`(x, y, 从起点到该点的总代价)`，优先按"总代价"从小到大排序（确保每次处理代价最小的节点）。  
   - 起点`(0,0)`初始代价为`0`，放入队列。

2. **处理节点**：  
   - 从队列中弹出当前代价最小的节点`(x,y)`。  
   - 若该节点已被访问（代价已确定），直接跳过（避免重复处理）。  
   - 标记该节点的最小代价为当前代价。若该节点是终点`(m-1, n-1)`，直接返回当前代价（因为Dijkstra保证此时的代价是最小的）。

3. **更新相邻节点**：  
   - 对当前节点`(x,y)`，尝试向**四个方向**（右、左、下、上，对应方向ID 1-4）移动，计算相邻节点`(nextX, nextY)`。  
   - 检查相邻节点是否越界（超出网格范围则跳过）。  
   - 计算移动到相邻节点的代价：若当前格子的默认方向与移动方向一致，代价不变；否则加`1`（修改方向的成本）。  
   - 若相邻节点未被访问，将其加入队列，等待处理。

## 五、代码实现

In [ ]:
import heapq

class State:
    def __init__(self, x, y, costFromStart):
        self.x = x  # 当前格子的行坐标
        self.y = y  # 当前格子的列坐标
        self.costFromStart = costFromStart  # 从起点(0,0)到当前格子的总代价

    def __lt__(self, other):
        # 重写小于运算符，让优先队列（最小堆）按代价从小到大排序
        return self.costFromStart < other.costFromStart

class Solution:
    def minCost(self, grid):
        return self.dijkstra(grid)
    
    def getDelta(self, directionId):
        # 1 -> 右 (x不变，y+1)，2 -> 左 (x不变，y-1)，3 -> 下 (x+1，y不变)，4 -> 上 (x-1，y不变)
        if directionId == 1:
            return [0, 1]
        if directionId == 2:
            return [0, -1]
        if directionId == 3:
            return [1, 0]
        return [-1, 0]
    
    def dijkstra(self, graph):
        m = len(graph)  # 网格行数
        n = len(graph[0])  # 网格列数
        distTo = [[-1] * n for _ in range(m)]  # 记录每个格子的最小代价，-1表示未访问

        pq = []  # 优先队列（最小堆）
        heapq.heappush(pq, State(0, 0, 0))  # 起点(0,0)初始代价0

        while pq:
            state = heapq.heappop(pq)  # 弹出当前代价最小的节点
            curX, curY = state.x, state.y
            curCost = state.costFromStart

            if distTo[curX][curY] != -1:  # 已访问过，跳过
                continue

            distTo[curX][curY] = curCost  # 标记当前格子的最小代价

            # 若到达终点，直接返回代价（Dijkstra保证此时是最小代价）
            if curX == m - 1 and curY == n - 1:
                return curCost

            # 尝试四个方向的移动（方向ID 1-4）
            for directionId in range(1, 5):
                delta = self.getDelta(directionId)  # 方向对应的坐标偏移
                nextX = curX + delta[0]
                nextY = curY + delta[1]

                # 检查是否越界
                if nextX < 0 or nextX >= m or nextY < 0 or nextY >= n:
                    continue

                # 计算移动到nextX, nextY的代价
                # 若当前格子的默认方向与移动方向一致，代价不变；否则+1（修改成本）
                nextCost = curCost
                if graph[curX][curY] != directionId:
                    nextCost += 1

                # 若相邻格子未访问，加入队列
                if distTo[nextX][nextY] == -1:
                    heapq.heappush(pq, State(nextX, nextY, nextCost))

        return -1  # 理论上不会走到这里，题目保证有解

## 六、测试代码

In [ ]:
# 测试用例1
grid1 = [[1,1,1,1],[2,2,2,2],[1,1,1,1],[2,2,2,2]]
solution = Solution()
result1 = solution.minCost(grid1)
print(f"测试用例1结果: {result1}")

# 测试用例2
grid2 = [[1,1,3],[3,2,2],[1,1,4]]
result2 = solution.minCost(grid2)
print(f"测试用例2结果: {result2}")

# 测试用例3
grid3 = [[1,2],[4,3]]
result3 = solution.minCost(grid3)
print(f"测试用例3结果: {result3}")

## 七、关键细节总结

1. **为什么遍历四个方向？**  
   因为可以修改格子的方向，所以即使原始方向是"右"，我们也可以选择向左、上、下移动（代价1），因此需要考虑所有可能的移动方向，才能找到最小代价路径。

2. **优先队列的作用**：  
   确保每次处理的是"当前代价最小的节点"，这是Dijkstra算法的贪心核心——一旦某个节点被弹出并标记，其代价就是最小的，不会有更优的路径再出现（因为边的代价非负）。

3. **distTo数组的作用**：  
   记录每个格子的最小代价，避免重复处理已确定代价的节点（优化效率）。

## 八、总结

本题通过将网格转化为"带权图"，将"最小修改代价"问题转化为"最短路径问题"，利用Dijkstra算法的贪心策略，每次处理代价最小的节点，最终高效找到从起点到终点的最小代价路径。核心是理解"方向修改的代价"如何转化为边的权重，以及Dijkstra算法在非负权图中的应用。